In [1]:
import tensorflow as tf
import os
import cv2
import numpy as np
from keras import Sequential
from keras.models import Model
from keras.layers import Dense, Flatten, Input, Concatenate
import matplotlib.pyplot as plt
import tqdm
from sklearn.preprocessing import LabelBinarizer
import pathlib

In [2]:
import pandas as pd
import xml.etree.ElementTree as ET

In [3]:
MMI_emotions =  pathlib.Path("C:/Users/cdr03/Documents/Thesis/dataset/MMI/Emotions")
MMI_FACS =  pathlib.Path("C:/Users/cdr03/Documents/Thesis/dataset/MMI/FACS")

In [4]:
def list_files_recursive(path, format):
    list_files = set()
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.startswith('S') and file.endswith(format) and "aucs" not in file:
                file_path = os.path.join(root, file)
                list_files.add(file_path)
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            list_files_recursive(dir_path, format)
    return list(list_files)


In [5]:
def create_dataframe_from_xml_files(path, format):
    xml_files = list_files_recursive(path, format)
    rows = []
    for file in xml_files:
        tree = ET.parse(file)
        root = tree.getroot()
        file_name = os.path.basename(file)
        clean_name = file_name.rsplit('.', 1)[0]
        emotion = int(root.find("./Metatag[@Name='Emotion']").attrib["Value"])
        if emotion < 7:
            rows.append({"Name": clean_name, "Emotion": int(emotion)})
    df = pd.DataFrame(rows, columns=["Name", "Emotion"])
    return df

In [194]:
MMI_emotion_labels_df = create_dataframe_from_xml_files(MMI_emotions, ".xml")

In [104]:
MMI_emotion_labels_df['Emotion']

0      6
1      1
2      1
3      4
4      4
      ..
208    2
209    4
210    1
211    4
212    1
Name: Emotion, Length: 213, dtype: int64

In [105]:
MMI_emotion_labels_df['Emotion'].isnull().values.any()

False

In [157]:
MMI_emotion_video_list = list_files_recursive(MMI_emotions, ".avi")

In [158]:
MMI_emotion_video_list

['C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1952\\S047-003.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1839\\S035-002.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\102\\S001-102.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1996\\S050-008.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1987\\S049-016.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1809\\S032-001.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1831\\S034-001.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\116\\S001-116.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\118\\S001-118.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1999\\S050-011.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1794\\S030-005.avi',
 'C:\\Users\\cdr03\\Documents\\Thesis\\dataset\\MMI\\Emotions\\1967\\S048-006.a

In [10]:
def image_to_array(img, size):
    img_array = np.array(img , dtype="float32")/255
    img_array = img_array.reshape(size, size, 1)
    # Return the numpy array
    return img_array

In [11]:
size = 224
def detect_and_crop_face(image, size):

    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Load the Haar Cascade classifier for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5)

    # Crop the image to contain the detected face (if any)
    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        face_image = image[y:y+h, x:x+w]
        if size is not None:
            face_image = cv2.resize(face_image, (size, size))
            face_image = image_to_array(face_image, size)
            not_detected_faces = None
    else:
        rotated=cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        rotated_faces = face_cascade.detectMultiScale(rotated, scaleFactor=1.1, minNeighbors=5)
        if len(rotated_faces) > 0:
            (x, y, w, h) = rotated_faces[0]
            face_image = rotated[y:y+h, x:x+w]

            face_image = cv2.resize(face_image, (size, size))
            face_image = image_to_array(face_image, size)
            not_detected_faces = None
        else:
            face_image = None
            not_detected_faces = image
            print("face wasn't detected")
    return face_image, not_detected_faces

In [12]:
def frame_generator(file_list):
    df = pd.DataFrame(columns=['Name','Data'])
    df_face_undetected = pd.DataFrame(columns=['Name','Data'])
    for file_path in tqdm.tqdm(file_list, desc="Processing files"):
        cap = cv2.VideoCapture(file_path)
        middle_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)/2)
        cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame)
        ret, frame = cap.read()
        if ret:
            file_name = os.path.basename(file_path)
            clean_name = file_name.rsplit('.', 1)[0]
            face_image_frame, non_faces = detect_and_crop_face(frame, size)
            temp_df = pd.DataFrame({'Name': [clean_name], 'Data':[face_image_frame]})
            df = pd.concat([df, temp_df], ignore_index=True)
            temp_df_undetected = pd.DataFrame({'Name': [clean_name], 'Data':[non_faces]})
            df_face_undetected = pd.concat([df_face_undetected, temp_df_undetected], ignore_index=True)

        cap.release()
    return df, df_face_undetected         



In [184]:
MMI_emotion_data, undedected_emotion = frame_generator(MMI_emotion_video_list)

Processing files:  85%|████████▍ | 200/236 [00:35<00:06,  5.60it/s]

face wasn't detected


Processing files: 100%|██████████| 236/236 [00:42<00:00,  5.54it/s]


In [188]:
cv2.imshow("test",MMI_emotion_data.Data.loc[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [195]:
MMI_Emotion_Middle_Frame = pd.merge(MMI_emotion_data,MMI_emotion_labels_df, how="inner", on=['Name'])

In [197]:
MMI_Emotion_Middle_Frame.to_pickle("./MMI_Emotion_Data_224")

FACS

In [32]:
from xml.dom import minidom

In [37]:
def create_dataframe_from_xml_au_files(path, format):
    xml_files = list_files_recursive(path, format)
    rows = []
    for file in xml_files:
        tree = minidom.parse(file)
        action_units = []
        file_name = os.path.basename(file)
        clean_name = file_name.rsplit('.', 1)[0]
        for action_unit in tree.getElementsByTagName('ActionUnit'):
            number = action_unit.attributes["Number"].value
            print(number)
            action_units.append(number)
        rows.append({"Name": clean_name, "ActionUnits": action_units})
    df = pd.DataFrame(rows, columns=["Name", "ActionUnits"])
    return df

In [38]:
MMI_FACS_Middle_Frame_label_df = create_dataframe_from_xml_au_files(MMI_FACS, ".xml")

1
2
5
25
26
45
18
45
14
17
23
24
25
26
45
1
2
5
12
25
26
45
1
2
5
25
45
4
26
8
25
26
36
10
12
23
25
29
30R
6
29
1
2
4
4
7
14
18
25
26
45
13
17
7
34
2
9
10
16
25
17
1
2
15
42
17
24
10
25
26
36
25
27
4
7
9
17
45
4
41
29
46
46L
6
18
20
25
42
6
13
25
45
14
34
26
36
17
26
28B
45
4
45
2
4
5
15
17
45
25
26
30
30R
6
12
25
6
13
26
36
45
27
1
2
20
25
45
5
25
26
30
30R
1
2
5
12
45
9
19
22
25
26
6
11
13
41
1
2
5
25
26
45
25
26
30R
4
45
7
9
45
44
45
1
4
7
15
17
14
33
13
17
24
43
25
26
30
30L
38
1
2
4
6
9
12
17
20
25
26
1
2
5
6
12
25
26
45
4
23
45
7
26
37
1
4
45
16
25
26
8
25
19
26
12
25
26
4
6
7
43
26
37
45
14
45
17
26
28
26
28
38
8
25
45
45
6
12
12
25
26
32
45
22
25
26
45
46
46R
2
4
5
10
16
20
25
26
45
18
45
19
23
26
45
17
24
45
15
45
2
5
16
25
26
45
1
2
5
25
26
30
1
2
5
10
15
17
45
1
4
18
6
12
25
25
27
9
10
25
26
43
25
26
32
26
37
45
4
7
10
25
26
4
46
46L
17
2
4
5
7
10
16
25
26
9
17
19
25
26
43
1
4
7
14
45
16
25
45
10
20
25
4
9
1
2
5
12
6
13
45
26
30
26
37
45
2
25
27
22
25
45
25
27
45
21
45
10
25

In [39]:
MMI_FACS_Middle_Frame_label_df['ActionUnits'].describe()

count         1162
unique         522
top       [26, 36]
freq            23
Name: ActionUnits, dtype: object

In [8]:
MMI_FACS_video_list = list_files_recursive(MMI_FACS, ".avi")

In [13]:
MMI_FACS_data, undedected_facs = frame_generator(MMI_FACS_video_list)

Processing files:  11%|█         | 124/1160 [00:31<04:23,  3.93it/s]

face wasn't detected


Processing files:  13%|█▎        | 154/1160 [00:39<04:12,  3.98it/s]

face wasn't detected


Processing files:  15%|█▍        | 173/1160 [00:44<04:23,  3.75it/s]

...

face wasn't detected


Processing files:  78%|███████▊  | 906/1160 [03:31<01:17,  3.30it/s]

face wasn't detected


Processing files: 100%|██████████| 1160/1160 [04:29<00:00,  4.30it/s]

face wasn't detected


In [207]:
cv2.imshow("test",MMI_FACS_data.Data.loc[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

: 

In [40]:
MMI_FACS_Middle_Frame = pd.merge(MMI_FACS_data,MMI_FACS_Middle_Frame_label_df, how="inner", on=['Name'])

In [41]:
MMI_FACS_Middle_Frame

,Name,Data,ActionUnits
0,S001-051,"[[[0.34117648], [0.3254902], [0.3254902], [0.3...","[29, 45]"
1,S004-084,"[[[0.11764706], [0.16470589], [0.21960784], [0...","[24, 34]"
2,S001-062,"[[[0.32156864], [0.32156864], [0.3137255], [0....","[33, 45]"
3,S005-016,"[[[0.3764706], [0.38039216], [0.42745098], [0....","[17, 20]"
4,S003-060,"[[[0.19607843], [0.2], [0.21568628], [0.223529...","[34, 38]"
...,...,...,...
1155,S004-056,"[[[0.07058824], [0.09019608], [0.13725491], [0...",[24]
1156,S002-054,"[[[0.37254903], [0.37254903], [0.36862746], [0...",[31]
1157,S040-012,"[[[0.6431373], [0.6431373], [0.6431373], [0.64...","[1, 2, 12, 25, 26, 45]"
1158,S004-063,"[[[0.34901962], [0.3529412], [0.34901962], [0....","[26, 36, 45, 28]"


In [42]:
MMI_FACS_Middle_Frame.dropna(inplace=True)

In [43]:
MMI_FACS_Middle_Frame

,Name,Data,ActionUnits
0,S001-051,"[[[0.34117648], [0.3254902], [0.3254902], [0.3...","[29, 45]"
1,S004-084,"[[[0.11764706], [0.16470589], [0.21960784], [0...","[24, 34]"
2,S001-062,"[[[0.32156864], [0.32156864], [0.3137255], [0....","[33, 45]"
3,S005-016,"[[[0.3764706], [0.38039216], [0.42745098], [0....","[17, 20]"
4,S003-060,"[[[0.19607843], [0.2], [0.21568628], [0.223529...","[34, 38]"
...,...,...,...
1154,S003-091,"[[[0.20784314], [0.21176471], [0.21568628], [0...","[46, 46R]"
1155,S004-056,"[[[0.07058824], [0.09019608], [0.13725491], [0...",[24]
1156,S002-054,"[[[0.37254903], [0.37254903], [0.36862746], [0...",[31]
1157,S040-012,"[[[0.6431373], [0.6431373], [0.6431373], [0.64...","[1, 2, 12, 25, 26, 45]"


In [44]:
MMI_FACS_Middle_Frame.to_pickle("./MMI_FACS_Data_224")

## Create training, validation and test datasets

In [2]:
from sklearn.model_selection import train_test_split

def create_stratified_datasets(X, y, test_size=0.1, val_size=0.1, random_state=None):
    """
    Creates three stratified datasets - train, validation, and test - from the input data X and target y.
    
    Parameters:
        X (array-like): The input data.
        y (array-like): The target variable.
        test_size (float): The proportion of the data to include in the test set.
        val_size (float): The proportion of the remaining data to include in the validation set.
        random_state (int): Seed for the random number generator.
    
    Returns:
        A tuple containing the train, validation, and test datasets, each as a tuple of input and target variables.
    """
    
    # split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    
    # split remaining data into validation and train sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size/(1-test_size), random_state=random_state, stratify=y_train)
    
    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

In [4]:
import pandas as pd
MMI_emotion = pd.read_pickle("./MMI_Emotion_Data_224")

In [5]:
MMI_emotion.dropna(inplace=True)

In [6]:
MMI_emotion.Emotion = MMI_emotion.Emotion.replace([2,3,4,5,6], [3,4,5,6,2])

In [7]:
MMI_emotion_target = MMI_emotion['Emotion']
MMI_data_target = MMI_emotion['Data']
(X_train, y_train), (X_val, y_val), (X_test, y_test) = create_stratified_datasets(MMI_data_target,MMI_emotion_target, random_state=12)

print(f'Distribution in training set: \n{y_train.value_counts().sort_index() / len(y_train)}\n\n'+
      f'Distribution in validation set: \n{y_val.value_counts().sort_index() / len(y_val)}\n\n'+
      f'Distribution in testing set: \n{y_test.value_counts().sort_index() / len(y_test)}')

Distribution in training set: 
1    0.154762
2    0.202381
3    0.154762
4    0.130952
5    0.202381
6    0.154762
Name: Emotion, dtype: float64

Distribution in validation set: 
1    0.190476
2    0.190476
3    0.142857
4    0.142857
5    0.190476
6    0.142857
Name: Emotion, dtype: float64

Distribution in testing set: 
1    0.136364
2    0.227273
3    0.136364
4    0.136364
5    0.227273
6    0.136364
Name: Emotion, dtype: float64


In [69]:
MMI_data_target

0      [[[0.22352941], [0.22745098], [0.24705882], [0...
1      [[[0.1882353], [0.19607843], [0.19607843], [0....
2      [[[0.26666668], [0.28235295], [0.28627452], [0...
3      [[[0.15686275], [0.15294118], [0.15294118], [0...
4      [[[0.24705882], [0.23137255], [0.25490198], [0...
                             ...                        
207    [[[0.7058824], [0.7058824], [0.69803923], [0.7...
208    [[[0.23921569], [0.25490198], [0.27058825], [0...
209    [[[0.14509805], [0.14901961], [0.14901961], [0...
210    [[[0.30588236], [0.31764707], [0.32941177], [0...
211    [[[0.18039216], [0.18039216], [0.1764706], [0....
Name: Data, Length: 211, dtype: object

In [8]:
print(f'Distribution in training set: \n{len(y_train)}\n\n'+
      f'Distribution in validation set: \n{len(y_val)}\n\n'+
      f'Distribution in testing set: \n{len(y_test)}')

Distribution in training set: 
168

Distribution in validation set: 
21

Distribution in testing set: 
22


In [9]:
X_train.to_pickle('./Clean_datasets/MMI/MMI_Emotion_X_train')
y_train.to_pickle('./Clean_datasets/MMI/MMI_Emotion_Y_train')
X_val.to_pickle('./Clean_datasets/MMI/MMI_Emotion_X_val')
y_val.to_pickle('./Clean_datasets/MMI/MMI_Emotion_Y_val')
X_test.to_pickle('./Clean_datasets/MMI/MMI_Emotion_X_test')
y_test.to_pickle('./Clean_datasets/MMI/MMI_Emotion_Y_test')

FACS

In [11]:
MMI_FACS_Middle_Frame = pd.read_pickle("./MMI_FACS_Data_224")

In [12]:
MMI_FACS_Middle_Frame['Data'].isnull().values.any()

False

In [29]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# fit the binarizer on the 'AU' column
mlb.fit(MMI_FACS_Middle_Frame['ActionUnits'])
# transform the 'AU' column into a one-hot encoded matrix
one_hot_encoded_au = mlb.transform(MMI_FACS_Middle_Frame['ActionUnits'])
# convert the matrix into a Pandas dataframe
one_hot_encoded_au_df = pd.DataFrame(one_hot_encoded_au, columns=mlb.classes_)
one_hot_encoded_au_df['Data'] = MMI_FACS_Middle_Frame['Data']

In [30]:
AU_columns = ['1', '2', '4', '5', '6', '9', '10', '12', '15', '17', '18', '20', '24', '25', '26', '28', 'Data']

In [31]:
pandas_df = one_hot_encoded_au_df[one_hot_encoded_au_df.columns.intersection(AU_columns)]

In [32]:
pandas_df.head(10)

,1,10,12,15,17,18,2,20,24,25,26,28,4,5,6,9,Data
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[[[0.34117648], [0.3254902], [0.3254902], [0.3..."
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"[[[0.11764706], [0.16470589], [0.21960784], [0..."
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[[[0.32156864], [0.32156864], [0.3137255], [0...."
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,"[[[0.3764706], [0.38039216], [0.42745098], [0...."
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[[[0.19607843], [0.2], [0.21568628], [0.223529..."
5,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,"[[[0.30588236], [0.3137255], [0.30980393], [0...."
6,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,"[[[0.3137255], [0.3019608], [0.28627452], [0.2..."
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[[[0.23137255], [0.23529412], [0.23137255], [0..."
8,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,"[[[0.26666668], [0.23137255], [0.22745098], [0..."
9,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,"[[[0.54509807], [0.4509804], [0.3372549], [0.2..."


In [34]:
MMI_facs_X = pandas_df['Data']

In [35]:
Au_labels_df = pandas_df.drop(columns = ['Data'])

In [36]:
all_zeros = Au_labels_df.eq(0).all(axis=1)
print(all_zeros.loc[all_zeros==True])
# display rows with only zeros
empty_rows = all_zeros.loc[all_zeros==True].index
print(empty_rows)

0       True
2       True
4       True
7       True
10      True
        ... 
1118    True
1119    True
1122    True
1143    True
1145    True
Length: 158, dtype: bool
Int64Index([   0,    2,    4,    7,   10,   12,   33,   38,   42,   50,
            ...
            1048, 1063, 1066, 1079, 1086, 1118, 1119, 1122, 1143, 1145],
           dtype='int64', length=158)


In [37]:
MMI_facs_X_clean = MMI_facs_X.drop(empty_rows)
Au_labels_df_clean  = Au_labels_df.drop(empty_rows)

In [38]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
import numpy as np
def multilable_split(X, y, size=0.1, random_state=12):
   msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=size, random_state=random_state)
   for train_index, test_index in msss.split(X, y):
      X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      y_train, y_test = y.iloc[train_index], y.iloc[test_index]
      return X_train, X_test, y_train, y_test

In [39]:
def create_balanced_datasets_multilabel(X, y, test_size=0.10, val_size=0.10):
    """
    Creates three stratified datasets - train, validation, and test - from the input data X and target y.
    
    Parameters:
        X (array-like): The input data.
        y (array-like): The target variable.
        test_size (float): The proportion of the data to include in the test set.
        val_size (float): The proportion of the remaining data to include in the validation set.
        random_state (int): Seed for the random number generator.
    
    Returns:
        A tuple containing the train, validation, and test datasets, each as a tuple of input and target variables.
    """
    # split data into train and test sets
    X_train, X_test, y_train, y_test = multilable_split(X, y, size = test_size)
    
    # split remaining data into validation and train sets
    X_train, X_val, y_train, y_val = multilable_split(X_train, y_train, size=val_size/(1-test_size))
    
    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

In [40]:
(X_train, y_train), (X_val, y_val), (X_test, y_test) = create_balanced_datasets_multilabel(MMI_facs_X_clean,Au_labels_df_clean)

In [41]:
print(f'Distribution in training set ({len(y_train)}): \n{y_train.value_counts().sort_index() / len(y_train)}\n\n'+
      f'Distribution in validation set ({len(y_val)}): \n{y_val.value_counts().sort_index() / len(y_val)}\n\n'+
      f'Distribution in testing set ({len(y_test)}): \n{y_test.value_counts().sort_index() / len(y_test)}')

Distribution in training set (798): 
1  10  12  15  17  18  2  20  24  25  26  28  4  5  6  9
0  0   0   0   0   0   0  0   0   0   0   0   0  0  0  1    0.003759
                                                    1  0    0.018797
                                                 1  0  0    0.012531
                                              1  0  0  0    0.033835
                                                       1    0.008772
                                                              ...   
1  1   0   0   0   0   1  1   0   1   1   0   1  0  1  1    0.001253
           1   0   0   0  0   0   0   0   0   1  0  0  0    0.001253
               1   0   1  0   0   0   0   0   0  1  0  0    0.001253
                                  1   0   0   0  1  0  0    0.001253
       1   0   0   0   1  0   0   1   1   0   0  0  1  0    0.002506
Length: 188, dtype: float64

Distribution in validation set (100): 
1  10  12  15  17  18  2  20  24  25  26  28  4  5  6  9
0  0   0   0   0   0  

In [42]:
X_train.to_pickle('./Clean_datasets/MMI/MMI_FACS_X_train')
y_train.to_pickle('./Clean_datasets/MMI/MMI_FACS_Y_train')
X_val.to_pickle('./Clean_datasets/MMI/MMI_FACS_X_val')
y_val.to_pickle('./Clean_datasets/MMI/MMI_FACS_Y_val')
X_test.to_pickle('./Clean_datasets/MMI/MMI_FACS_X_test')
y_test.to_pickle('./Clean_datasets/MMI/MMI_FACS_Y_test')